In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq

openjdk-8-jdk-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1PD8MJcdYUOQ4N6DfZKZSfk-wsivqWXgV'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('c1.txt')

id='1WmboygIOkHQbUMvDjTpTilfd7D9qPi5q'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('data.txt')

In [5]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf


# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [6]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.mllib.linalg import Vectors

In [12]:
###DATA FORMATTING FOR SPARK

data = sc.textFile("data.txt").collect()
for i in range(len(data)):
  data[i]=data[i].split(" ")
for i in range(len(data)):
  for j in range(len(data[i])):
    data[i][j]=float(data[i][j])
for i in range(len(data)):
  data[i]=str(data[i])

data = sc.parallelize(data).map(Vectors.parse)

centers = sc.textFile("c1.txt").collect()
for i in range(len(centers)):
  centers[i]=centers[i].split(" ")
for i in range(len(centers)):
  for j in range(len(centers[i])):
    centers[i][j]=float(centers[i][j])

In [13]:
initialModel = KMeansModel([Vectors.parse(str(v)) for v in centers])
print("iteration 1 :","{:.6f}".format(initialModel.computeCost(data)))

model = KMeans.train(data, 10, 1, initializationMode="k-means", initialModel=initialModel)
print("iteration 2 :","{:.6f}".format(model.computeCost(data)))

for i in range(8):
  model = KMeans.train(data, 10, 1, initializationMode="k-means", initialModel=model)
  print("iteration",i+3, ":", "{:.6f}".format(model.computeCost(data)))

iteration 1 : 623660345.306411
iteration 2 : 509862908.297545
iteration 3 : 485480681.872008
iteration 4 : 463997011.685013
iteration 5 : 460969266.572996
iteration 6 : 460537847.982769
iteration 7 : 460313099.653545
iteration 8 : 460003523.889408
iteration 9 : 459570539.317735
iteration 10 : 459021103.342291
